#### THE LIFE CYCLE OF A SPARK APPLICATION (INSIDE SPARK)

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option","some-value").getOrCreate()

In [3]:
df1 = spark.range(2, 10000000, 2)

In [4]:
df2 = spark.range(2, 10000000, 4)

In [5]:
step1 = df1.repartition(5)

In [6]:
step12 = df2.repartition(6)

In [7]:
step2 = step1.selectExpr("id * 5 as id")

In [8]:
step3 = step2.join(step12,["id"])

In [9]:
step4 = step3.selectExpr("sum(id)")

In [10]:
step4.collect()

[Row(sum(id)=2500000000000)]

In [11]:
step4.explain()

== Physical Plan ==
*HashAggregate(keys=[], functions=[sum(id#8L)])
+- Exchange SinglePartition
   +- *HashAggregate(keys=[], functions=[partial_sum(id#8L)])
      +- *Project [id#8L]
         +- *SortMergeJoin [id#8L], [id#3L], Inner
            :- *Sort [id#8L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#8L, 200)
            :     +- *Project [(id#0L * 5) AS id#8L]
            :        +- Exchange RoundRobinPartitioning(5)
            :           +- *Range (2, 10000000, step=2, splits=Some(1))
            +- *Sort [id#3L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#3L, 200)
                  +- Exchange RoundRobinPartitioning(6)
                     +- *Range (2, 10000000, step=4, splits=Some(1))
